In [11]:
# Install spaCy 
!pip3 install -U spacy==3.3.0

# Download the large English model for spaCy
!python3 -m spacy download en_core_web_lg

# Install textacy which will also be useful
#!pip3 install -U textacy

     |████████████████████████████████| 6.2 MB 4.7 MB/s            
     |████████████████████████████████| 10.2 MB 36.2 MB/s            
     |████████████████████████████████| 181 kB 68.5 MB/s            
     |████████████████████████████████| 662 kB 56.2 MB/s            
     |████████████████████████████████| 42 kB 2.2 MB/s             
     |████████████████████████████████| 9.9 MB 55.0 MB/s            
     |████████████████████████████████| 127 kB 63.5 MB/s            
     |████████████████████████████████| 456 kB 56.8 MB/s            
     |████████████████████████████████| 58 kB 746 kB/s             
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocor

In [16]:
from sagemaker import get_execution_role
import boto3
import spacy

In [20]:
bucket='riyab'
sub='load'

conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=sub)['Contents']
for f in contents:
    print(f['Key'])

load/
load/text (1).txt


In [21]:

# Connect to S3 bucket and download file
s3 = boto3.resource('s3')
buck=s3.Bucket(bucket)
# for obj in buck.objects.all():
#     key = obj.key
#     text = obj.get()['Body'].read()
# text=text.decode("utf-8")

In [24]:
my_bucket = 'riyab'
my_file = 'load/text (1).txt'
s3client = boto3.client('s3')
response = s3client.get_object(Bucket=my_bucket, Key=my_file)
body = response['Body'].read()
text=body.decode('cp1252')
text

"Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

In [25]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# Parse the text with spaCy. This runs the entire pipeline.
# so we've already segmented and tokenized the doc in one line
doc = nlp(text)

#with this pipeline we can look at different kind of words and phrases
#for instance let us look at flat phrases that have a noun as their head
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Lorem Ipsum Ipsum nsubj is
dummy text text attr is
the printing and typesetting industry industry pobj of
Lorem Ipsum Ipsum nsubj been
the industry's standard dummy text text attr been
an unknown printer printer nsubj took
a galley galley dobj took
type type pobj of
it it dobj scrambled
a type specimen book book dobj make
It It nsubj survived
not only five centuries centuries dobj survived
the leap leap conj centuries
electronic typesetting typesetting pobj into
It It nsubjpass popularised
the 1960s 1960s pobj in
the release release pobj with
Letraset sheets sheets pobj of
Lorem Ipsum passages passages dobj containing
desktop publishing software software pobj with
Aldus PageMaker PageMaker pobj like
versions versions pobj including
Lorem Ipsum Ipsum pobj of


In [26]:
#as sentences

sents=text.split('.')
sents

['Lorem Ipsum is simply dummy text of the printing and typesetting industry',
 " Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book",
 ' It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged',
 ' It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum',
 '']

In [27]:
#as words

for word in doc:
    print(word.text)

Lorem
Ipsum
is
simply
dummy
text
of
the
printing
and
typesetting
industry
.
Lorem
Ipsum
has
been
the
industry
's
standard
dummy
text
ever
since
the
1500s
,
when
an
unknown
printer
took
a
galley
of
type
and
scrambled
it
to
make
a
type
specimen
book
.
It
has
survived
not
only
five
centuries
,
but
also
the
leap
into
electronic
typesetting
,
remaining
essentially
unchanged
.
It
was
popularised
in
the
1960s
with
the
release
of
Letraset
sheets
containing
Lorem
Ipsum
passages
,
and
more
recently
with
desktop
publishing
software
like
Aldus
PageMaker
including
versions
of
Lorem
Ipsum
.


In [28]:
# now we'll filter out stop words
filter_doc=""
for token in doc:
  if token.is_stop:
    print(token.text)
  else:
    filter_doc+=token.text+ " "

is
of
the
and
has
been
the
's
ever
since
the
when
an
a
of
and
it
to
make
a
It
has
not
only
five
but
also
the
into
It
was
in
the
with
the
of
and
more
with
of


In [29]:
#Since spacy does not have a built in lemmatizer, we'd build a lemmatizer instead

#Text Lemmatization: finding the base word of our word in doc
# for instance stand becomes standing, fell becomes fall
# we'll put lemmatized words in a new filtered doc
doc = nlp(filter_doc)
filter_doc=""
for token in doc:
  filter_doc+=token.lemma_ + " "
  if token.text != token.lemma_:
    print(token.text+" becomes "+token.lemma_)

1500s becomes 1500
took becomes take
scrambled becomes scramble
survived becomes survive
centuries becomes century
remaining becomes remain
popularised becomes popularise
1960s becomes 1960
sheets becomes sheet
containing becomes contain
passages becomes passage
including becomes include
versions becomes version


In [30]:
!mkdir results

In [31]:
import numpy as np
doc = nlp(filter_doc)
words=[]
with open('results/words.txt','w') as f:

    for word in doc:
        words.append(word.text)
        f.write(word.text)
        f.write('\n')
np.save('results/words.npy',words,allow_pickle=True)

In [32]:
import pickle
pickle.dump(words, open('results/words.pickle', 'wb'))

In [33]:
s3 = boto3.client('s3')

folder_name = "results"

s3.put_object(Bucket=my_bucket, Key=(folder_name+'/'))

{'ResponseMetadata': {'RequestId': '50A6RNSBMZ2H1HCX',
  'HostId': 'gNQYz/u1NGUT0PDzAW/yzXFqo1MAObk87hCD60LQQTZhDQl8jFvupbyKKq4Y55AuUzKWm1MwQnI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'gNQYz/u1NGUT0PDzAW/yzXFqo1MAObk87hCD60LQQTZhDQl8jFvupbyKKq4Y55AuUzKWm1MwQnI=',
   'x-amz-request-id': '50A6RNSBMZ2H1HCX',
   'date': 'Mon, 06 Jun 2022 08:15:03 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [34]:
with open('results/words.npy','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.npy'),Body=data)
with open('results/words.txt','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.txt'),Body=data)
with open('results/words.pickle','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.pickle'),Body=data)